# manually check all segmentationmap

In [9]:
import  numpy               as     np
from    scripts.tools       import *
from    scripts.masking     import *
from    astropy.table       import Table
from    astropy.io          import fits
import astropy.units        as     u
from   astropy.cosmology    import Planck18
from    tqdm                import tqdm
import  os
import  matplotlib.pyplot   as     plt
import  time
%matplotlib auto



Using matplotlib backend: macosx


In [ ]:

obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    if obj['manual_select'] == 'selected': continue
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        seg     = find_data('SEG',hdu)[1]
        if find_data('SEG_MOD',hdu) != None: seg = find_data('SEG_MOD',hdu)[1]
        seg.data = np.where(seg.data == obj['ID'],obj['ID'],0)
        line_maps = [find_data('DSCI',hdu)[1],
                     find_data('LINE_HA',hdu)[1],
                     find_data('LINE_HB',hdu)[1]]
        seg_var = run_segmentation_editor(line_maps,seg,obj['ID'])
        seg_var.name = 'SEG_MOD'
        fits.ImageHDU(data=seg_var.data,header=seg_var.header).writeto(f"seg/{file_name(obj,'seg')}",overwrite=True)
        hdu = save_update(seg_var,hdu)
        hdu.flush()

  0%|          | 0/1024 [00:00<?, ?it/s]

  0%|          | 1/1024 [00:21<6:01:04, 21.18s/it]

  0%|          | 2/1024 [00:25<3:08:25, 11.06s/it]

  0%|          | 3/1024 [00:34<2:52:31, 10.14s/it]

  0%|          | 4/1024 [00:37<2:08:27,  7.56s/it]

...

  5%|▍         | 51/1024 [07:02<3:31:02, 13.01s/it]

  5%|▌         | 53/1024 [07:05<2:04:19,  7.68s/it]

  5%|▌         | 54/1024 [07:11<2:09:15,  8.00s/it]


TypeError: 'NoneType' object is not subscriptable

: 

#save segs

In [3]:
from    astropy.table       import Table
from    astropy.io          import fits
from    tqdm                import tqdm
from    scripts.tools       import *
obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    try:
        path = f"data_extracted/{file_name(obj,prefix='extracted')}"
        path_seg = f"seg/{file_name(obj,'seg')}"
        with fits.open(path,mode='update') as hdu:
            with fits.open(path_seg) as seg:
                seg[1].name = 'SEG_MOD'
                to_save = fits.ImageHDU(data = seg[1].data,name='SEG_MOD')
                save_update(to_save,hdu)
                hdu.flush()
    except Exception as e:
            print(f"! {obj['subfield']}-{obj['ID']} failed, error{e}")

100%|██████████| 158/158 [01:15<00:00,  2.09it/s]


In [8]:
cat_lis = Table.read('full_object_catalog.fits')
cat_hasline = cat_lis[np.logical_and(cat_lis['Ha_FLUX'] >1, cat_lis['Hb_FLUX']>1)]
sn_ha = cat_hasline['Ha_FLUX']/cat_hasline['Ha_FLUX_ERR']
cat_hasline['sn_ha'] = sn_ha
sn_hb = cat_hasline['Hb_FLUX']/cat_hasline['Hb_FLUX_ERR']
cat_hasline['sn_hb'] = sn_hb
selection = np.logical_and(sn_ha>0,sn_hb>0)
obj_lis = cat_hasline[selection]

manual_selection = np.array((
    ['ERSPRIME',40759],
    ['ERSPRIME',43823],
    ['ERSPRIME',45646],
    ['GN1',37031],
    ['GN1',37913],
    ['GN1',37395],
    ['GN1',37623],
    ['GN1',37738],
    ['GN1',38134],
    ['GN2', 11228],
    ['GN2', 14895],
    ['GN2', 14895],
    #['GN2', 16752],
    ['GN2', 17579],
    ['GN2', 17829],
    ['GN2', 18197],
    ['GN2', 18224],
    ['GN2', 18315],
    ['GN2', 21552],
    #['GN3', 28121],
    ['GN3', 30204],
    ['GN3', 32166],
    ['GN3', 32660],
    ['GN3', 34570],
    ['GN3', 34708],
    ['GN3', 34838],
    ['GN3', 35042],
    ['GN3', 35568],
    ['GN3', 35822],
    ['GN3', 19075],
    ['GN3', 28121],
    #['GN4', 21690],
    ['GN4', 24377],
    #['GN4', 24582],
    ['GN4', 26015],
    #['GN4', 28379],
    ['GN4', 27282],
    ['GN5', 31789],
    ['GN7', 12769],
    #['GN7', 13909],
    ['GN7', 13197],
    #['GN7', 13777],
    #['GN7', 14281],
    ['GN7', 14716],
    #['GN7', 14184],
    #['GN7', 14850],
    ['GN7', 15127],
    ['GN7', 15204],
    ['GN7', 15300],
    ['GN7', 17352],
    ['GN7', 17532],
    ['GN7', 22746],
    ['GN7', 19504],
    ['GN7', 23580],
    ['GS1', 45789],
    ['GS1', 47214],
    #['GS1', 47399],
    ['GS1', 47375],
    ['GS1', 48850],
    #['GS2', 48123],
    ['GS1', 49063],
    #['GS1', 36182],
    ['GS2', 45633],
    ['GS2', 46564],
    ['GS2', 47770],
    ['GS2', 45795],
    #['GS3', 37010],
    ['GS2', 48652],
    ['GS3', 34363],
    ['GS3', 37903],
    ['GS3', 40611],
    ['GS3', 41370],
    ['GS4', 20698],
    ['GS4', 24803],
    ['GS4', 24947],
    ['GS4', 25641],
    #['GS4', 25745],
    ['GS4', 25592],
    ['GS4', 26739],
    ['GS4', 29686],
    ['GS4', 20970],
    ['GS2', 46564],
    #['GS4', 22730],
    ['GS4', 26175],
    #['GS4', 26807],
    ['GS4', 29717],
    ['GS4', 29846],
    ['GS4', 29394],
    ['GS4', 28190],
    ['GS4', 30976],
    ['GS4', 31454],
    ['GS5', 38513],
    ['GS4', 20976],
    ['GS5', 41122],
    #['GS5', 44066],
    #['GS5', 40976],
    #['GS5', 44519],
    ['GS5', 42758],
    #['GS5', 42472],
))

is_selected = np.isin(obj_lis['ID'],manual_selection[:,1]) & np.isin(obj_lis['subfield'],manual_selection[:,0])
obj_lis['manual_select'] = np.where(is_selected,'selected','keep') 
obj_lis['pixel_length']  =  np.deg2rad(0.1/3600) * Planck18.angular_diameter_distance(obj_lis['z_MAP']).to(u.kpc).value

obj_lis.write('obj_lis_selected.fits',overwrite=True)

print('total num of objs in the fields',len(cat_lis))
print('num of objs after s/n selection:',len(obj_lis))
print('num of obj after agn selection',len(obj_lis[obj_lis['tag']!='agn']))
print('num of obj after manual and agn selection',len(obj_lis[obj_lis['manual_select']!='selected']))

total num of objs in the fields 6048
num of objs after s/n selection: 1041
num of obj after agn selection 979
num of obj after manual and agn selection 968
